In [ ]:
# summarizes statistics by subdomain (avg bias, rmse, etc)
# from the model results output by the 'analyze.py' procedure in pyap
# GO 20230613
# data in: CTDcast_metrics_hindcast.pickle (output from pyap analyze.py - not scores.py)
#       - note the above must be generated w/ mods made to pyap by GO
# data out: csv of summary stats (means by subdomain and depth class)

In [1]:
import scipy.io
import os
import pickle
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import sys
import math

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [12]:
# code to check length (num of records)

# data output as pickle files by pyap
# path1 = 'D:/temp_nemo/RUN203_PLOTS_SCORES/'
path1 = 'D:/temp_nemo/RUN203_PLOTS_SCORES/'
path1 = 'D:/temp_nemo/RUN100/'

# pic_data = "CTDcast_metrics_hindcast.pickle"
pic_data = "CTDcast_metrics_hindcast.pickle"
#pic_data = "SST_class4_SalishSea1500-RUN203.pickle"
scores = pickle.load(open(os.path.join(path1,pic_data), 'rb'))
# modelrun = 'SalishSea1500-RUN203'
modelrun = 'SalishSea1500-RUN203'
modelrun = 'SalishSea1500-RUN100'

In [19]:
print(scores[modelrun]['class4'])

{'Nanoose_1002_CastCTD_2017-10-02_1448h': None, 'Nanoose_1002_CastCTD_2017-10-02_2331h': None, 'Nanoose_1003_CastCTD_2017-10-03_1502h': None, 'Nanoose_1003_CastCTD_2018-10-03_1624h': None, 'Nanoose_1004_CastCTD_2016-10-04_1703h': None, 'Nanoose_1004_CastCTD_2017-10-04_1510h': None, 'Nanoose_1004_CastCTD_2017-10-04_2140h': None, 'Nanoose_1004_CastCTD_2018-10-04_1534h': None, 'Nanoose_1005_CastCTD_2017-10-05_1514h': None, 'Nanoose_1005_CastCTD_2017-10-05_2153h': None, 'Nanoose_1006_CastCTD_2017-10-06_1515h': None, 'Nanoose_1012_CastCTD_2017-10-12_1549h': None, 'Nanoose_1016_CastCTD_2018-10-16_1730h': None, 'Nanoose_1016_CastCTD_2018-10-16_2038h': None, 'Nanoose_1017_CastCTD_2017-10-17_1635h': None, 'Nanoose_1019_CastCTD_2017-10-19_1544h': None, 'Nanoose_1020_CastCTD_2017-10-20_1516h': None, 'Nanoose_1020_CastCTD_2017-10-20_1526h': None, 'Nanoose_1024_CastCTD_2017-10-24_1526h': None, 'Nanoose_1024_CastCTD_2017-10-24_1534h': None, 'Nanoose_1025_CastCTD_2017-10-25_1523h': None, 'Nanoose_102

In [37]:
scores[modelrun]['scores']['Nanoose_0_CastCTD_1991-01-03_1804h']['bias'].keys()

dict_keys(['Full', '0->10', '10->30', '30->50', '50->100', '100->200', '200->400', '>400'])

In [85]:
limitn = 100000 # for debug
scores_data = scores[modelrun]['scores']
sdomains = ['DI','SGN','SGS','GI','HS','JFS','PS']
stats=['bias', 'rmse', 'crmse', 'skill1981', 'mean_obs', 'mean_mod', 'std_obs', 'std_mod']
dep_levs = ['Full', '0->10', '10->30', '30->50', '50->100', '100->200', '200->400', '>400']
vars_ts = ['T','S']

def get_mean_CTD(scores_data, var, sdomain, stat, dep_lev):
    avg = []
    avg_n = 0
    n = 0
    bad = 0
    for ctd in scores[modelrun]['scores'].keys(): 
        if not scores[modelrun]['scores'][ctd]:
            n+=1
            continue
        if not scores[modelrun]['scores'][ctd][stat][dep_lev][var]:
            bad+=1
            continue
        else:
         
            if scores[modelrun]['scores'][ctd]['SubDomain']==sdomain:
                sval = scores[modelrun]['scores'][ctd][stat][dep_lev][var]
                
                if not math.isnan(sval):
                    avg.append(sval)
                    avg_n += 1
        n+=1
        if n > limitn:
            break
    
#     print(var, sdomain, stat, dep_lev)
#     print("bad ", bad)
    if avg_n == 0:
        avg_all = np.nan
    elif avg_n != 1:
        avg_all = np.round(np.nanmean(avg),3)
    
    return avg_all, avg_n

row_labels = ["sd_levs"]
for sdomain in sdomains:
    for dep_lev in dep_levs:
        row_labels.append(sdomain + "_" + dep_lev)
# print((row_labels))

all_means_cols = []
cnt_n_col = ["n obs"]
i = 0
for var in vars_ts:
    print(var)
    for stat in stats:
        print(stat)
        one_mean_col = [var + "_" + stat]
        for sdomain in sdomains: 
            avg = []
            avg_n = 0
            for dep_lev in dep_levs:
                avg, avg_n = get_mean_CTD(scores_data, var, sdomain, stat, dep_lev)
                one_mean_col.append(avg)
                if i == 0:
                    cnt_n_col.append(avg_n)
        if len(all_means_cols) != 0:
            all_means_cols = np.column_stack((all_means_cols, one_mean_col))
        else: 
            all_means_cols = row_labels
            all_means_cols = np.column_stack((all_means_cols,cnt_n_col))
            all_means_cols = np.column_stack((all_means_cols,one_mean_col))
        i+=1
# print(all_means_cols)
# print(avg, avg_n)
print(pd.DataFrame(all_means_cols))      
pd.DataFrame(all_means_cols).to_csv(os.path.join(path1,modelrun+"_meanmetricssummary.csv"))


T
bias
rmse
crmse
skill1981
mean_obs
mean_mod
std_obs
std_mod
S
bias
rmse
crmse
skill1981
mean_obs
mean_mod
std_obs
std_mod
              0      1       2       3        4            5           6   \
0        sd_levs  n obs  T_bias  T_rmse  T_crmse  T_skill1981  T_mean_obs   
1        DI_Full   3649    0.16   0.945    0.621        0.607      10.388   
2       DI_0->10   3624   0.404   1.231    0.527        0.348      11.173   
3      DI_10->30   3580  -0.055   0.681    0.243        0.403      10.164   
4      DI_30->50   3080   0.014   0.488    0.085        0.271       9.593   
5     DI_50->100   2334   0.281   0.403    0.175        0.312       9.228   
6    DI_100->200   1800   0.399   0.424    0.051        0.199       9.269   
7    DI_200->400    892   0.463   0.468    0.036        0.105       9.286   
8        DI_>400      0     nan     nan      nan          nan         nan   
9       SGN_Full  12365    0.04   0.755    0.619        0.846      10.358   
10     SGN_0->10  12325  -0.0